# Chess Bot Implementation

Trivial example that moves a single piece


In [4]:
import numpy as np
import os

from pydrake.all import (
    ConstantVectorSource,
    DiagramBuilder,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    MultibodyPlant,
    Parser,
    PiecewisePolynomial,
    PiecewiseQuaternionSlerp,
    RigidTransform,
    RollPitchYaw,
    RotationMatrix,
    Simulator,
    Solve,
    StartMeshcat,
    TrajectorySource,
)
from pydrake.multibody import inverse_kinematics
from pydrake.trajectories import PiecewisePolynomial

from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.scenarios import AddMultibodyTriad, MakeManipulationStation
from manipulation.utils import running_as_notebook

In [3]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [36]:
meshcat.Delete()
def setup_manipulation_station():
    builder = DiagramBuilder()
    with open('board.dmd.yaml', 'r') as f:
        scenario = f.read()
    station = builder.AddSystem(
        MakeManipulationStation(
            scenario,
            time_step=1e-3,
        )
    )
    plant = station.GetSubsystemByName("plant")
    scene_graph = station.GetSubsystemByName("scene_graph")
    AddMultibodyTriad(plant.GetFrameByName("body"), scene_graph)

    MeshcatVisualizer.AddToBuilder(
        builder,
        station.GetOutputPort("query_object"),
        meshcat,
        MeshcatVisualizerParams(delete_on_initialization_event=False),
    )

    gripper_position = builder.AddSystem(ConstantVectorSource([0.1]))
    builder.Connect(
        gripper_position.get_output_port(), station.GetInputPort("wsg_position")
    )

    diagram = builder.Build()

    context = plant.CreateDefaultContext()
    gripper = plant.GetBodyByName("body")

    initial_pose = plant.EvalBodyPoseInWorld(context, gripper)

    simulator = Simulator(diagram)
    simulator.set_target_realtime_rate(1.0)
    simulator.AdvanceTo(0.1)

    return initial_pose

setup_manipulation_station()

RuntimeError: GetBodyByName(): There is no Body named 'BlackKnight' anywhere in the model (valid names are: Board, base, body, iiwa_link_0, iiwa_link_1, iiwa_link_2, iiwa_link_3, iiwa_link_4, iiwa_link_5, iiwa_link_6, iiwa_link_7, left_finger, piece_body, right_finger, table_top_link, world)